## Imports

In [1]:
# Imports
from spark_config import init_spark
import os
import glob
from datetime import datetime

## Start Spark Session

In [2]:
spark = init_spark("Pipeline VACUUM Delta")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Define Delta Table Paths

In [3]:
# Configurações de caminho
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

## Define Functions

In [4]:
# Função para localizar diretórios Delta (procura por _delta_log)
def find_delta_paths(root_path):
    paths = []
    if not os.path.exists(root_path):
        return paths
    for candidate in glob.glob(os.path.join(root_path, "**"), recursive=True):
        if os.path.isdir(candidate) and os.path.exists(os.path.join(candidate, "_delta_log")):
            paths.append(os.path.abspath(candidate))
    # remove duplicatas e ordena
    return sorted(list(dict.fromkeys(paths)))

# Função para executar VACUUM em uma lista de caminhos
def vacuum_paths(paths, retention_hours=168, dry_run=True):
    """
    Executa VACUUM para cada caminho em `paths`.
    - retention_hours: horas a reter (padrão 168 = 7 dias)
    - dry_run: se True apenas imprime os comandos
    """
    for p in paths:
        cmd = f"VACUUM delta.`{p}` RETAIN {retention_hours} HOURS"
        print(f"[{datetime.now()}] Comando: {cmd}")
        if dry_run:
            print(f"  -> dry-run: não executado")
            continue
        try:
            spark.sql(cmd)
            print(f"  ✅ Vacuum finalizado: {p}")
        except Exception as e:
            print(f"  ❌ Falha ao executar VACUUM em {p}: {e}")

## Main

In [ ]:
# Monta lista de caminhos padrão (gold + silver) e adiciona caminhos explícitos comuns
paths = []
paths.extend(find_delta_paths(base_gold_path))
paths.extend(find_delta_paths(base_silver_path))

if not paths:
    print("Nenhum diretório Delta encontrado nas pastas gold/silver especificadas.")
else:
    print("Caminhos Delta detectados para vacuum:")
    for p in paths:
        print(" -", p)

# Execução padrão: dry-run. Para execução real, chamar com dry_run=False
vacuum_paths(paths, retention_hours=168, dry_run=False)

print("\nObservação: Para executar o VACUUM real, re-execute a célula chamando:\n    vacuum_paths(paths, retention_hours=168, dry_run=False)\n")

Caminhos Delta detectados para vacuum:
 - D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_ativo_financeiro
 - D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_cliente
 - D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_tempo
 - D:\Projetos\Jornada_financas_pessoais\data\delta\gold\fato_carteira
 - D:\Projetos\Jornada_financas_pessoais\data\delta\gold\fato_cotacao
 - D:\Projetos\Jornada_financas_pessoais\data\delta\silver\stg_controle_ativo
 - D:\Projetos\Jornada_financas_pessoais\data\delta\silver\stg_cotacao_historica
[2025-12-20 18:09:48.478917] Comando: VACUUM delta.`D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_ativo_financeiro` RETAIN 168 HOURS
  ✅ Vacuum finalizado: D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_ativo_financeiro
[2025-12-20 18:11:17.212038] Comando: VACUUM delta.`D:\Projetos\Jornada_financas_pessoais\data\delta\gold\dim_cliente` RETAIN 168 HOURS
  ✅ Vacuum finalizado: D:\Projetos\Jornada_financas_pessoais\data\de

## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()
print("\n✅ Spark finalizado")


✅ Spark finalizado
